In [22]:
import os

from nuscenes.nuscenes import NuScenes
from utils.custom_lidar_api import CustomLidarApi
from utils.custom_map_api_expansion import CustomNuScenesMap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

import torch
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline 

In [2]:
batch_size = 4
learning_rate = 0.001

class_names = [ 'road_block', 'walkway', 'road_divider', 'traffic_light']

In [3]:
class_dict = dict()

for i, name in enumerate(class_names):
    class_dict[name] = i
class_array = np.eye(len(class_names))

In [4]:
locations = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
version = 'v1.0-trainval'
dataroot = 'E:/datasets/nuscenes'

In [5]:
nusc = NuScenes(version=version, dataroot=dataroot, verbose=True)
ldr_api = CustomLidarApi(nusc)

map_api = dict([])
for location in locations:
    map_api[location] = CustomNuScenesMap(dataroot = dataroot, map_name= location)

Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
Done loading in 61.722 seconds.
Reverse indexing ...
Done reverse indexing in 9.0 seconds.


In [6]:
# get all sample token
sample_tokens = []
for scene in nusc.scene:
    token = scene['first_sample_token']
    while token != scene['last_sample_token']:
        sample_tokens.append(token)
        sample = nusc.get('sample', token)
        token = sample['next']
print(len(sample_tokens))

33299


In [7]:
def train_val_split(sample_tokens, ratio = 0.1, shuffle = True):
    index = np.array(range(len(sample_tokens)))
    index = np.random.choice(index.shape[0], index.shape[0], replace = False)
    
    valid_num = int(index.shape[0] * ratio)
    
    valid_tokens = sample_tokens[:valid_num]
    train_tokens = sample_tokens[valid_num:]
    
    return train_tokens, valid_tokens

train_tokens, valid_tokens = train_val_split(sample_tokens)

print(len(sample_tokens))
print(len(train_tokens))
print(len(valid_tokens))
    

33299
29970
3329


In [12]:
class NusceneDataset(Dataset):
    def __init__(self, tokens, nusc, map_api, ldr_api, train = True):
        self.tokens = tokens
        self.nusc = nusc
        self.map_api = map_api
        self.ldr_api = ldr_api
        self.train = train
        
        self.length = len(tokens)
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        token = self.tokens[idx]
        
        sample = self.nusc.get('sample', token)
        scene = self.nusc.get('scene', sample['scene_token'])
        log_meta = self.nusc.get('log', scene['log_token'])
        
        print(scene)
        location = log_meta['location']
        sample_data = self.nusc.get('sample_data', sample['data']['LIDAR_TOP'])
        
        pc = self.ldr_api.get_lidar_from_keyframe(token, car_coord = True)
        ego = self.ldr_api.get_egopose_from_keyframe(token)
        cs = self.nusc.get('calibrated_sensor', sample_data['calibrated_sensor_token'])
        structures = self.map_api[log_meta['location']].get_closest_structures(class_names, ego, global_coord=False)
        
        X = pc.points
        if self.train:
            classes, objects = self.get_label(structures)
        else:
            classes, objects = self.get_label(list())
        
        return X, classes, objects
    
    def get_label(self, structures):
        classes = torch.zeros((64, 4))
        objects = torch.zeros((64,30,2))
        
        for i, struct in enumerate(structures):
            clas = struct['class']
            nodes = struct['nodes']
            
            classes[i,:] = torch.Tensor(class_array[class_dict[clas], :])
            objects[i,:,:] = torch.Tensor(nodes.reshape(1, 30, 2))
        
        return classes, objects
        
            
        

train_dataset = NusceneDataset(tokens = train_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)
valid_dataset = NusceneDataset(tokens = valid_tokens, nusc = nusc, map_api = map_api, ldr_api = ldr_api)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)

In [14]:
for X, c, y in train_loader:
    print(X.shape)
    print(c.shape)
    print(y.shape)

{'token': '7ed1b613f0ee47ceaf6d19ca3947f825', 'log_token': 'f93e8d66ce4b4fbea7062d19b1fe29fb', 'nbr_samples': 39, 'first_sample_token': 'c4048cf79e5c4b88a5845dfc98b3b1e9', 'last_sample_token': 'd501737a45d146f8af94511adaf27761', 'name': 'scene-1109', 'description': 'Night, rain, bus, lightning'}
{'token': '4efbf4c0b77f467385fc2e19da45c989', 'log_token': 'ee155e99938a4c2698fed50fc5b5d16a', 'nbr_samples': 40, 'first_sample_token': '637fc435f95f4bc1b72f09811f2d78d2', 'last_sample_token': 'ce7c414656da4ddc90b8f1b82f1169d8', 'name': 'scene-0963', 'description': 'Motorcycle, wait at intersection, cones, cars, bus'}
{'token': '04219bfdc9004ba2af16d3079ecc4353', 'log_token': '5bd40b613ac740cd9dbacdfbc3d68201', 'nbr_samples': 41, 'first_sample_token': '73c335a2a207457ea36cc855b2417bd9', 'last_sample_token': 'bb12ced8c5ac4dfbaa4347a9cb7dbb35', 'name': 'scene-0922', 'description': 'Buses, peds'}
{'token': '2fc3753772e241f2ab2cd16a784cc680', 'log_token': '3a43824b84534c98bda1d07548db5817', 'nbr_sa

FileNotFoundError: [Errno 2] No such file or directory: 'E:/datasets/nuscenes\\samples/LIDAR_TOP/n008-2018-08-31-11-19-57-0400__LIDAR_TOP__1535729151048569.pcd.bin'

In [28]:
scene_names = []
for token in sample_tokens:
        sample = nusc.get('sample', token)
        sample_data = nusc.get('sample_data', sample['data']['LIDAR_TOP'])
        
        if os.path.isfile(os.path.join(nusc.dataroot, sample_data['filename'])):
            pass
        else:
            scene_name = nusc.get('scene', sample['scene_token'])['name']
            scene_names.append(int(scene_name[-4:]))


[220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 221, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 225, 226, 226, 226, 226, 226, 226, 226,

In [29]:
print(set(scene_names))

{220, 221, 222, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 315, 316, 317, 318, 321, 323}
